In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc
from joblib import dump

pd.options.display.max_columns=500
pd.options.display.max_rows=50

os.getcwd()

'/mnt/drv5/jian/predictive_model'

In [2]:
df_input=pd.read_csv("./Data_tabel_WithOut_DCM_20190831_with_competitors_JL_2020-05-23.csv")
print(df_input.shape)

list_cols=df_input.columns.tolist()
df_input.columns=[x.strip() for x in list_cols]

# df_input.columns.tolist()
df_input.head(2)


(1047882, 453)


,customer_id_hashed,DV_single_week_1,DV_single_week_2,DV_single_week_3,DV_single_week_4,DV_cumulative_week_1,DV_cumulative_week_2,DV_cumulative_week_3,DV_cumulative_week_4,sign_up_date,sign_up_location,weeks_since_sign_up,P_zip,S_zip,else_10_zip,signed_online,distc_to_sign_up,email_unsub_19Oct,nearest_BL_store,nearest_BL_dist,week_diff,total_trans_since_registration,total_items,total_sales,trans_in_store,unique_stores,trans_online,department_minus_one_trans,department_108_trans,department_109_trans,department_110_trans,department_111_trans,department_114_trans,department_115_trans,department_120_trans,department_130_trans,department_140_trans,department_150_trans,department_160_trans,department_170_trans,department_210_trans,department_230_trans,department_250_trans,department_270_trans,department_310_trans,department_320_trans,department_330_trans,department_340_trans,department_350_trans,department_351_trans,department_352_trans,department_353_trans,department_354_trans,department_355_trans,department_360_trans,department_361_trans,department_362_trans,department_363_trans,department_364_trans,department_365_trans,department_366_trans,department_367_trans,department_370_trans,department_410_trans,department_425_trans,department_470_trans,department_480_trans,department_510_trans,department_520_trans,department_521_trans,department_526_trans,department_530_trans,department_540_trans,department_550_trans,department_560_trans,department_608_trans,department_610_trans,department_612_trans,department_615_trans,department_710_trans,department_800_trans,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,total_units_department_minus_1_1st_trans,total_units_department_108_1st_trans,total_units_department_109_1st_trans,total_units_department_110_1st_trans,total_units_department_111_1st_trans,total_units_department_114_1st_trans,total_units_department_115_1st_trans,total_units_department_120_1st_trans,total_units_department_130_1st_trans,total_units_department_140_1st_trans,total_units_department_150_1st_trans,total_units_department_160_1st_trans,total_units_department_170_1st_trans,total_units_department_210_1st_trans,total_units_department_230_1st_trans,total_units_department_250_1st_trans,total_units_department_270_1st_trans,total_units_department_310_1st_trans,total_units_department_320_1st_trans,total_units_department_330_1st_trans,total_units_department_340_1st_trans,total_units_department_350_1st_trans,total_units_department_351_1st_trans,total_units_department_352_1st_trans,total_units_department_353_1st_trans,total_units_department_354_1st_trans,total_units_department_355_1st_trans,total_units_department_360_1st_trans,total_units_department_361_1st_trans,total_units_department_362_1st_trans,total_units_department_363_1st_trans,total_units_department_364_1st_trans,total_units_department_365_1st_trans,total_units_department_366_1st_trans,total_units_department_367_1st_trans,total_units_department_370_1st_trans,total_units_department_410_1st_trans,total_units_department_425_1st_trans,total_units_department_470_1st_trans,total_units_department_480_1st_trans,total_units_department_510_1st_trans,total_units_department_520_1st_trans,total_units_department_521_1st_trans,total_units_department_526_1st_trans,total_units_department_530_1st_trans,total_units_department_540_1st_trans,total_units_department_550_1st_trans,total_units_department_560_1st_trans,total_units_department_608_1st_trans,total_units_department_610_1st_trans,total_units_department_612_1st_trans,total_units_department_615_1st_trans,total_units_department_710_1st_trans,total_units_department_800_1st_trans,week_recent_1_trans,week_counts_to_now_recent_one,label_1_recent_1_trans_also_1st,purchase_channel_1st_trans_recent_one,total_sales_recent_1_trans,total_units_recent_1_trans,total_units_department_minus_1_trans_recent_one,total_units_department_108_trans_recent_one,total_units_department_109_trans_recent_one,total_units_department_110_

In [3]:
# from the iteration of col below
cols_remove_rows=['nearest_BL_dist',
                  'Consumer Goods Rental_All Other Consumer Goods Rental',
                  'Consumer Goods Rental_Consumer Electronics and Appliances Rental',
                  'Department Stores_Department Stores',
                  'Furniture Stores_Furniture Stores',
                  'Grocery Stores_Convenience Stores',
                  'Grocery Stores_Supermarkets and Other Grocery (except Convenience) Stores',
                  'Home Furnishings Stores_All Other Home Furnishings Stores',
                  'Lawn and Garden Equipment and Supplies Stores_Nursery, Garden Center, and Farm Supply Stores',
                  'Office Supplies, Stationery, and Gift Stores_Gift, Novelty, and Souvenir Stores',
                  'Other Miscellaneous Store Retailers_All Other Miscellaneous Store Retailers (except Tobacco Stores)',
                  'Other Miscellaneous Store Retailers_Pet and Pet Supplies Stores',
                  'Specialty Food Stores_All Other Specialty Food Stores',
                  'Sporting Goods, Hobby, and Musical Instrument Stores_Hobby, Toy, and Game Stores']
# week_recent_2_trans: make sure to have at least 2 transactions
for col in cols_remove_rows:
    df_input=df_input[pd.notnull(df_input[col])]
    
print(df_input.shape)

(994565, 453)


In [4]:
cols_include_TBD=['email_unsub_19Oct','nearest_BL_dist']

cols_to_delete=['customer_id_hashed','DV_single_week_1','DV_single_week_2','DV_single_week_3','DV_single_week_4',
                'customer_zip_code','week_1st_trans','week_recent_1_trans','week_recent_2_trans','week_recent_3_trans','week_recent_4_trans','week_recent_5_trans']
for col in cols_to_delete:
    del df_input[col]
'''    
list_col_recent_345=[x for x in df_input.columns.tolist() if ("recent_three" in x) or ("recent_3" in x) or ("recent_four" in x) or ("recent_4" in x) or ("recent_five" in x) or ("recent_5" in x)]
for col in list_col_recent_345:
    del df_input[col]
'''    


'    \nlist_col_recent_345=[x for x in df_input.columns.tolist() if ("recent_three" in x) or ("recent_3" in x) or ("recent_four" in x) or ("recent_4" in x) or ("recent_five" in x) or ("recent_5" in x)]\nfor col in list_col_recent_345:\n    del df_input[col]\n'

In [5]:
list_1st_col_for_each_recent_week=["week_counts_to_now_recent_one","week_counts_to_now_recent_two","week_counts_to_now_recent_three",
                                  "week_counts_to_now_recent_four","week_counts_to_now_recent_five"]
list_last_col_for_each_recent_week=["total_units_department_800_1st_trans_recent_one","total_units_department_800_recent_2_trans",
                                    "total_units_department_800_recent_3_trans","total_units_department_800_recent_4_trans",
                                    "total_units_department_800_recent_5_trans"]

ind_start_competitor=424

In [7]:
df_input.columns.tolist()

['DV_cumulative_week_1',
 'DV_cumulative_week_2',
 'DV_cumulative_week_3',
 'DV_cumulative_week_4',
 'sign_up_date',
 'sign_up_location',
 'weeks_since_sign_up',
 'P_zip',
 'S_zip',
 'else_10_zip',
 'signed_online',
 'distc_to_sign_up',
 'email_unsub_19Oct',
 'nearest_BL_store',
 'nearest_BL_dist',
 'week_diff',
 'total_trans_since_registration',
 'total_items',
 'total_sales',
 'trans_in_store',
 'unique_stores',
 'trans_online',
 'department_minus_one_trans',
 'department_108_trans',
 'department_109_trans',
 'department_110_trans',
 'department_111_trans',
 'department_114_trans',
 'department_115_trans',
 'department_120_trans',
 'department_130_trans',
 'department_140_trans',
 'department_150_trans',
 'department_160_trans',
 'department_170_trans',
 'department_210_trans',
 'department_230_trans',
 'department_250_trans',
 'department_270_trans',
 'department_310_trans',
 'department_320_trans',
 'department_330_trans',
 'department_340_trans',
 'department_350_trans',
 'departm

In [6]:
dict_dfs={}

for i_with_recent_weeks in range(5):
    str_key="RW_"+str(i_with_recent_weeks+1)
    col_start=list_1st_col_for_each_recent_week[i_with_recent_weeks]
    col_end=list_last_col_for_each_recent_week[i_with_recent_weeks]
    
    ind_start=df_input.columns.tolist().index(col_start)
    ind_end=df_input.columns.tolist().index(col_end)
    
    list_cols_needed=list(range(ind_end+1))
    list_competitors=list(range(424,len(df_input.columns.tolist())))
    df=df_input.iloc[:,list_cols_needed+list_competitors]
    df=df[pd.notnull(df[col_start])]
    for col in df.columns.tolist():
        if "_department_" in col:
            df[col]=df[col].fillna(0)
        else:
            df_col=df[[col]]
            if df_col[pd.isnull(df_col[col])].shape[0]>0:
                print("still existing nan col:",col)
    dict_dfs.update({str_key:df})
    print("shape of df saved:",df.shape)

ValueError: 'total_units_department_800_1st_trans_recent_one' is not in list

# Build Model

In [ ]:
# f1 & confusion matrics -- https://en.wikipedia.org/wiki/F1_score
# solvers --  https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
# psuedo r2 -- https://datascience.oneoffcoder.com/psuedo-r-squared-logistic-regression.html
# id selection -- https://www.kaggle.com/residentmario/automated-feature-selection-with-sklearn
# sklearn feature selection for Log_R -- https://towardsdatascience.com/a-look-into-feature-importance-in-logistic-regression-models-a4aa970f9b0f
# p values for sklearn -- https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold

from sklearn import metrics
import matplotlib.pyplot as plt
import sklearn


print(sklearn.__version__)

In [ ]:
folder_model_output="./output_modeling_WO_DCM_"+str(datetime.datetime.now().date())+"/"

try:
    os.stat(folder_model_output)
except:
    os.mkdir(folder_model_output)
    

In [ ]:
def process_from_dict_to_XY_list(dict_all_inputs,n_DV,n_RW,ptcg_test=0.25,r=0.98):
    # dict_all_inputs: the dict of each recent weeks available
    # r: the univariate without enough variance
    
    # output_list: [X_train_scaled,X_test_scaled,y_train,y_test]
    key_rw="RW_"+str(n_RW)
    df=dict_all_inputs[key_rw]
    
    col_needed_X=[x for x in df.columns.tolist() if x[:3]!="DV_"]
    col_Y="DV_cumulative_week_updated_"+str(n_DV)
    
    df_X=df[col_needed_X]
    df_Y=df[col_Y].tolist()

    list_cols_X=df_X.columns.tolist()
    
    r=0.98 # ivs with >= r same values will be removed
    threshold_variance_iv=r*(1-r)
    sel = VarianceThreshold(threshold=threshold_variance_iv)
    df_X_reduced=sel.fit_transform(df_X)
    
    print("df_X_reduced.shape after removing the low variance ivs",df_X_reduced.shape)
    del df_X_reduced

    indices = [i for i, x in enumerate(list(sel.get_support())) if x == True]
    list_cols_X_r=df_X.iloc[1:,indices].columns.tolist()

    df_X=df_X.iloc[:,indices]

    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.25, random_state=42)

    X_train_scaled=scale(X_train)
    X_test_scaled=scale(X_test)
    
    print("X_train_scaled.shape",X_train_scaled.shape)
    print("X_test_scaled.shape",X_test_scaled.shape)
    
    list_dfs_XC_XT_yC_yT=[X_train_scaled,X_test_scaled,y_train,y_test]
    return list_dfs_XC_XT_yC_yT, list_cols_X_r

In [ ]:
def generate_step_table_of_test_mode(model,x_test_input,y_test_input,threshold_list = [(x+1)/100 for x in range(0,100)]):
    y_test_true = y_test_input
    list_prob=[x[0] for x in model.predict_proba(x_test_input)]
    df_output=pd.DataFrame()
    for i in threshold_list:
        y_test_pred=[1 if x<=i else 0 for x in list_prob]

        accuracy_score = metrics.accuracy_score(y_test_true,y_test_pred)    
        tn, fp, fn, tp = metrics.confusion_matrix(y_test_true, y_test_pred).ravel()
        # 
        TPR=tp/(tp+fn) #recall
        FNR=fn/(tp+fn)
        FPR=fp/(fp+tn)
        TNR=tn/(fp+tn)

        PPV=tp/(tp+fp) #precission
        f1_score = 2*(TPR*PPV)/(TPR+PPV)

        df=pd.DataFrame({"predicted_positive":len([x for x in y_test_pred if x==1]),
                         "predicted_negative":len([x for x in y_test_pred if x==0]),
                         "accuracy_score":accuracy_score,
                         'true_negative':tn,
                         'false_positive_(Type1_Error)':fp,
                         'fale_negative_(Type2_Error)':fn,
                         'true_positive':tp,
                         'true_positive_rate':TPR,
                         'false_negative_rate':FNR,
                         'false_positive_rate':FPR,
                         'true_negative_rate':TNR,
                         'precission_(Positive predictive value)':PPV,
                         'f1_score':f1_score
                        },index=[i])
        df_output=df_output.append(df)
    return df_output

In [ ]:
def run_p_values(model,input_x,input_y):
    from scipy import stats
    # https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
    params = np.append(model.intercept_,model.coef_)
    predictions = model.predict(input_x)

    newX = pd.DataFrame({"Constant":np.ones(len(input_x))}).join(pd.DataFrame(input_x))
    MSE = (sum((input_y-predictions)**2))/(len(newX)-len(newX.columns))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    df_output_p = pd.DataFrame()
    df_output_p["Coefficients"],df_output_p["Standard Errors"],df_output_p["t values"],df_output_p["Probabilities"] = [params,sd_b,ts_b,p_values]
    return df_output_p


In [ ]:
def run_log_regression_model(input_x_train,input_y_train,input_x_test,input_y_test,n_week,solver="saga",C=1,penalty="l1"):
    # n_week 1-4
    # solver and penalty: https://scikit-learn.org/stable/modules/linear_model.html
    # solver: [‘liblinear’,‘lbfgs’,‘newton-cg’,‘sag’,‘saga’]

    
    selector = SelectFromModel(estimator=LogisticRegression()).fit(input_x_train, input_y_train)
    
    print("selector.threshold_",selector.threshold_)
    list_support_boolean=selector.get_support()
    
    x_train_selected=selector.transform(input_x_train)
    x_test_selected=selector.transform(input_x_test)
    
    list_selected_features= [i for i, x in enumerate(list_support_boolean) if x == True]
    list_selected_features= [list_cols_X_r[i] for i in list_selected_features]
    print("len(list_selected_features)",len(list_selected_features))

    mod_log_r=LogisticRegression(random_state=0,solver=solver,C=C,penalty='l1').fit(x_train_selected,input_y_train)
    
    df_p_value=run_p_values(model=mod_log_r,input_x=x_train_selected,input_y=input_y_train)
    
    df_step_table=generate_step_table_of_test_mode(model=mod_log_r,x_test_input=x_test_selected,y_test_input=input_y_test)

    df_coefficient=pd.DataFrame({"coefficient":mod_log_r.coef_[0]},index=list_selected_features).reset_index().rename(columns={"index":"IVs"})
    df_intercept=pd.DataFrame({"coefficient":list(mod_log_r.intercept_)},index=['intercept']).reset_index().rename(columns={"index":"IVs"})
    df_coefficient=df_intercept.append(df_coefficient)
    
    # plot the ROC 
    img_roc=metrics.plot_roc_curve(mod_log_r, x_test_selected, input_y_test)
    plt.show()
    
    df_coefficient=df_coefficient.reset_index()
    del df_coefficient['index']
    df_p_value=df_p_value.reset_index()
    del df_p_value['index']
    df_coefficient=pd.concat([df_coefficient,df_p_value],axis=1)

    list_y_train_pred=mod_log_r.predict(x_train_selected)
    
    return mod_log_r,df_step_table,df_coefficient,list_y_train_pred,img_roc
    

In [ ]:
for m in range(5):
    m_RW=m+1
    writer=pd.ExcelWriter(folder_model_output+"Model_WithOut_DCM_output_with_trans_RW_available_for_%s_JL_%s.xlsx"%(str(m+1),str(datetime.datetime.now().date())),engine="xlsxwriter")
    for n in range(4):
        n_DV_week=n+1
        
        list_dfs_XC_XT_yC_yT,list_cols_X_r=process_from_dict_to_XY_list(dict_all_inputs=dict_dfs,
                                              n_DV=n_DV_week,
                                              n_RW=m_RW,ptcg_test=0.25,r=0.98)
        
        print(m_RW,n_DV_week)
        model,mod_step_tables,mod_coefficient_df,mode_list_y_train_pred,img_roc=run_log_regression_model(
            input_x_train=list_dfs_XC_XT_yC_yT[0],
            input_y_train=list_dfs_XC_XT_yC_yT[2],
            input_x_test=list_dfs_XC_XT_yC_yT[1],
            input_y_test=list_dfs_XC_XT_yC_yT[3],n_week=n_DV_week)
        
        dump(model, folder_model_output+'model_WithOut_DCM_%s_DV%s.joblib'%(m_RW,str(n_DV_week)))
        
        mod_coefficient_df.to_excel(writer,"Coeff_%s_%s"%(str(m_RW),str(n_DV_week)),index=False)
        mod_step_tables.to_excel(writer,"Step_%s_%s"%(str(m_RW),str(n_DV_week)),index=True)
        
        # worksheet = writer.sheets["Step_%s_%s"%(str(m_RW),str(n_DV_week))]
        
        # worksheet.insert_image("H1",img_roc)

    writer.save()


In [ ]:
mod_coefficient_df.head(2)

In [ ]:
# Efron’s 𝑅2
def efron_rsquare(y, y_pred):
    n = float(len(y))
    t1 = np.sum(np.power(y - y_pred, 2.0))
    t2 = np.sum(np.power((y - (np.sum(y) / n)), 2.0))
    return 1.0 - (t1 / t2)

# fun for the mcfadeen r2
def full_log_likelihood(w, X, y):
    score = np.dot(X, w).reshape(1, X.shape[0])
    return np.sum(-np.log(1 + np.exp(score))) + np.sum(y * score)

def null_log_likelihood(w, X, y):
    z = np.array([w if i == 0 else 0.0 for i, w in enumerate(w.reshape(1, X.shape[1])[0])]).reshape(X.shape[1], 1)
    score = np.dot(X, z).reshape(1, X.shape[0])
    return np.sum(-np.log(1 + np.exp(score))) + np.sum(y * score)

# McFadden’s 𝑅2
def mcfadden_rsquare(w, X, y):
    return 1.0 - (full_log_likelihood(w, X, y) / null_log_likelihood(w, X, y))
# McFadden’s Adjusted 𝑅2
def mcfadden_adjusted_rsquare(w, X, y):
    k = float(X.shape[1])
    return 1.0 - ((full_log_likelihood(w, X, y) - k) / null_log_likelihood(w, X, y))

# McKelvey & Zavoina 𝑅2
def mz_rsquare(y_pred):
    return np.var(y_pred) / (np.var(y_pred) + (np.power(np.pi, 2.0) / 3.0) )

# Count 𝑅2
def get_num_correct(y, y_pred, t=0.5):
    y_correct = np.array([0.0 if p < t else 1.0 for p in y_pred])
    return sum([1.0 for p, p_pred in zip(y, y_correct) if p == p_pred])

def count_rsquare(y, y_pred, t=0.5):
    n = float(len(y))
    num_correct = get_num_correct(y, y_pred, t)
    return num_correct / n

# Adjust count 𝑅2
def get_count_most_freq_outcome(y):
    num_0 = 0
    num_1 = 0
    for p in y:
        if p == 1.0:
            num_1 += 1
        else:
            num_0 += 1
    return float(max(num_0, num_1))

def count_adjusted_rsquare(y, y_pred, t=0.5):
    correct = get_num_correct(y, y_pred, t)
    total = float(len(y))
    n = get_count_most_freq_outcome(y)
    return (correct - n) / (total - n)